# Finding Similar Items: Textually Similar Documents

In [40]:
from numpy import unique
from pandas import read_csv
url = 'https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv'
df = read_csv(url)

In [41]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [42]:
df_len_10 = df['text'].iloc[:10] # First 10 corpus
df_len_10

0    tv future in the hands of viewers with home th...
1    worldcom boss  left books alone  former worldc...
2    tigers wary of farrell  gamble  leicester say ...
3    yeading face newcastle in fa cup premiership s...
4    ocean s twelve raids box office ocean s twelve...
5    howard hits back at mongrel jibe michael howar...
6    blair prepares to name poll date tony blair is...
7    henman hopes ended in dubai third seed tim hen...
8    wilkinson fit to face edinburgh england captai...
9    last star wars  not for children  the sixth an...
Name: text, dtype: object

A class **Shingling** that constructs k–shingles of a given length k (e.g., 10) from a given document, computes a hash value for each unique shingle, and represents the document in the form of an ordered set of its hashed k-shingles.

In [43]:
import random, math
class Shingling:
   
    def __init__(self, input_text, input_k):
        self.text = input_text
        self.k = input_k
           
    def shingles(self):
        for line in self.text:
            return {line[i:i + self.k] for i in range(len(line) - self.k + 1)} 
    
    def ordered_hash(self, input_sh):
        random.seed(111)
        a = random.randint(0,100)
        b = random.randint(0,100)        
        # Convert string into int
        return set(sorted(set((a*(int.from_bytes(x.encode(), 'little')) + 
                           b)%1039205197 for x in input_sh)))

In [44]:
import io
sample_text = io.StringIO(df_len_10[0])
# Construct 10-shingles
sh = Shingling(sample_text,10) 
shingles = sh.shingles() 
# Ordered set of hashed 100-shingles
ordered_set = sh.ordered_hash(shingles) 

In [45]:
sets = []
for i in range(2):
    sample_text = io.StringIO(df_len_10[i])
    # Construct 10-shingles
    sh = Shingling(sample_text,10) 
    shingles = sh.shingles() 
    # Ordered set of hashed 100-shingles
    ordered_set = sh.ordered_hash(shingles)
    sets.append(ordered_set)

A class **CompareSets** that computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.

A class **MinHashing** that builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).

In [46]:
class MinHashing:

    def __init__(self, input_set, input_n):
        self.s = input_set
        self.n = input_n

    def hashing(self):
        min_hash = [None] * self.n # Vector of length n
        random.seed(111)
        for n in range(self.n):
            a = random.randint(0, 2**32-1) # Max shingle id
            b = random.randint(0, 2**32-1)
            vector = [None] * len(self.s) # Vector of length set hashed shingles
            for i, value in enumerate(self.s):
                hashcode = ((a * value + b) % 4294967311) # Make sure that prime > max shingle id
                vector[i] = hashcode 
            min_hash[n] = min(vector)
        return min_hash
    

In [47]:
mh = MinHashing(ordered_set,20)
minhash = mh.hashing()

A class **CompareSignatures** that estimates similarity of two integer vectors – minhash signatures – as a fraction of components, in which they agree.

In [48]:
class CompareSignatures:
    def __init__(self, vector1, vector2):
        self.v1 = vector1
        self.v2 = vector2

    def estimate(self):
        count = 0
        for i, elem in enumerate(self.v1):
            if self.v1[i] == self.v2[i]: # Assumed that length of both vectors are the same
                count += 1

        return count/len(self.v1)

In [49]:
mh1 = MinHashing(sets[0],20).hashing()
mh2 = MinHashing(sets[1],20).hashing()

In [51]:
CompareSignatures(mh1,mh1).estimate()

1.0

In [52]:
CompareSignatures(mh1,mh2).estimate()

0.0

(Optional task for extra 2 bonus) A class **LSH** that implements the LSH technique: given a collection of minhash signatures (integer vectors) and a similarity threshold t, the LSH class (using banding and hashing) finds candidate pairs of signatures agreeing on at least fraction t of their components.